In [1]:
from enzy_pred import *

import warnings
warnings.filterwarnings("ignore")

logger.setLevel(logging.INFO)

2022-04-19 21:51:13,64 tensorflow DEBUG Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
Using TensorFlow backend.
2022-04-19 21:51:14,673 matplotlib DEBUG CONFIGDIR=/home/cz332/.config/matplotlib
2022-04-19 21:51:14,678 matplotlib DEBUG (private) matplotlib data path: /home/cz332/miniconda3/envs/big-data-env/lib/python3.6/site-packages/matplotlib/mpl-data
2022-04-19 21:51:14,686 matplotlib DEBUG matplotlib data path: /home/cz332/miniconda3/envs/big-data-env/lib/python3.6/site-packages/matplotlib/mpl-data
2022-04-19 21:51:14,687 matplotlib DEBUG loaded rc file /home/cz332/miniconda3/envs/big-data-env/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
2022-04-19 21:51:14,692 matplotlib DEBUG matplotlib version 3.2.2
2022-04-19 21:51:14,692 matplotlib DEBUG interactive is False
2022-04-19 21:51:14,694 matplotlib DEBUG platform is linux
2022-04-19 21:51:14,694 matplotlib DEBUG loaded modules: ['builtin

In [2]:
self = EnzymePrediction(nBits=128)
self.main()

In [3]:
self.x_ec_rxn_bert_fps()

In [4]:
# self.train()
self.imbalance_train()

In [5]:
self.model.main()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 864)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               442880    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 256)               3302

KeyboardInterrupt: 

In [ ]:
self = self.model

In [ ]:
self.y = self.df.y

In [ ]:
# self.df = self.df[self.df.columns[0:264]]

In [ ]:
self.df['y'] = self.y

In [ ]:
self.df

In [ ]:
self.split_data()
self.scale_data()

In [ ]:
self.df

In [ ]:
from keras import layers

In [ ]:
input_dim = self.df_train_0_x_rescaled.shape[1]  # num of predictor variables,

input_layer = Input(shape=(input_dim,))
encoder = layers.BatchNormalization()(input_layer)
encoder = layers.Dense(500,
                activation='elu',
                activity_regularizer=regularizers.l1(1e-3))(encoder)
encoder = layers.Dense(300, activation="elu")(encoder)
encoder = layers.Dense(300, activation="elu")(encoder)
encoder = layers.Dense(200, activation="elu")(encoder)
encoder = layers.Dense(200, activation="elu")(encoder)
encoder = layers.Dense(100, activation="elu")(encoder)
encoder = layers.Dense(100, activation="elu")(encoder)
encoder = layers.Dense(50, activation="elu")(encoder)
encoder = layers.Dense(50, activation="elu")(encoder)
encoder = layers.Dense(20, activation="elu")(encoder)
decoder = layers.Dense(50, activation="elu")(encoder)
decoder = layers.Dense(50, activation="elu")(decoder)
decoder = layers.Dense(100, activation="elu")(decoder)
decoder = layers.Dense(100, activation="elu")(decoder)
decoder = layers.Dense(200, activation="elu")(decoder)
decoder = layers.Dense(200, activation="elu")(decoder)
decoder = layers.Dense(300, activation="elu")(decoder)
decoder = layers.Dense(300, activation="elu")(decoder)
decoder = layers.Dense(500, activation="elu")(decoder)
decoder = layers.Dense(input_dim, activation="sigmoid")(decoder)
self.autoencoder = Model(inputs=input_layer, outputs=decoder)
self.autoencoder.summary()

self.autoencoder.compile(metrics=['accuracy'],
                         loss='mean_squared_error',
                         optimizer='adam')

self.cp = ModelCheckpoint(filepath="autoencoder_classifier_20220317.h5",
                          save_best_only=True,
                          verbose=0)

self.tb = TensorBoard(log_dir='./logs',
                      histogram_freq=0,
                      write_graph=True,
                      write_images=True)

self.history = self.autoencoder.fit(self.df_train_0_x_rescaled, self.df_train_0_x_rescaled,
                                    epochs=self.nb_epoch,
                                    batch_size=self.batch_size,
                                    shuffle=True,
                                    validation_data=(self.df_valid_0_x_rescaled, self.df_valid_0_x_rescaled),
                                    verbose=1,
                                    callbacks=[self.cp, self.tb]).history

In [ ]:
self.valid_x_predictions = self.autoencoder.predict(self.df_valid_x_rescaled)
mse = np.mean(np.power(self.df_valid_x_rescaled - self.valid_x_predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': self.df_valid['y']})
precision_rt, recall_rt, threshold_rt = precision_recall_curve(error_df.True_class, error_df.Reconstruction_error)
plt.plot(threshold_rt, precision_rt[1:], label="Precision",linewidth=5)
plt.plot(threshold_rt, recall_rt[1:], label="Recall",linewidth=5)
plt.title('Precision and recall for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Precision/Recall')
plt.legend()
plt.show()

In [ ]:
error_df[error_df.True_class==0].describe()

In [ ]:
error_df[error_df.True_class==1].describe()

In [ ]:
pd.DataFrame(data={'1': self.df_valid_x_rescaled[0], '2':self.valid_x_predictions[0]})[0:50]